In [1]:
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

dataset_path = Path("datasets/raw")

def default_loader(path):
    return Image.open(path).convert('RGB')

class SkillDataset(Dataset):
    def __init__(self, path: Path) -> None:
        super().__init__()
        all_img = list(dataset_path.glob("*.png"));
        self.c0 = [p for p in all_img if p.stem.endswith("_0")]
        self.c1 = [p for p in all_img if p.stem.endswith("_1")]
        self.c2 = [p for p in all_img if p.stem.endswith("_2")]
        self.c3 = [p for p in all_img if p.stem.endswith("_3")]
        print("len c0: ", len(self.c0), " c1: ", len(self.c1), " c2: ", len(self.c2), " c3: ", len(self.c3))

        self.loader = default_loader
        self.transform = transforms.Compose([
            transforms.GaussianBlur(3, sigma=(0.1, 2.0)),
            transforms.RandomPosterize(3),
            transforms.RandomAdjustSharpness(3),
            transforms.RandomAutocontrast(),
            transforms.ToTensor(),
        ])
        # 图片没多大，一次性全部载入内存算了
        self.data = [ self.get(i) for i in range(len(self))]
    
    def __len__(self):
        return len(self.c0) + len(self.c1) + len(self.c2) + len(self.c3)
    
    def get(self, index):
        if index % 1000 == 0:
            print("load ", index, " / ", len(self))
        if index < len(self.c0):
            return self.transform(self.loader(self.c0[index])), 0
        elif index < len(self.c0) + len(self.c1):
            return self.transform(self.loader(self.c1[index - len(self.c0)])), 1
        elif index < len(self.c0) + len(self.c1) + len(self.c2):
            return self.transform(self.loader(self.c2[index - len(self.c0) - len(self.c1)])), 2
        else:
            return self.transform(self.loader(self.c3[index - len(self.c0) - len(self.c1) - len(self.c2)])), 3
    
    def __getitem__(self, index):
        return self.data[index][0], self.data[index][1]
        

dataset = SkillDataset(dataset_path)

e:\ENV\anaconda\envs\chat\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


len c0:  4865  c1:  4703  c2:  12248  c3:  4404
load  0  /  26220
load  1000  /  26220
load  2000  /  26220
load  3000  /  26220
load  4000  /  26220
load  5000  /  26220
load  6000  /  26220
load  7000  /  26220
load  8000  /  26220
load  9000  /  26220
load  10000  /  26220
load  11000  /  26220
load  12000  /  26220
load  13000  /  26220
load  14000  /  26220
load  15000  /  26220
load  16000  /  26220
load  17000  /  26220
load  18000  /  26220
load  19000  /  26220
load  20000  /  26220
load  21000  /  26220
load  22000  /  26220
load  23000  /  26220
load  24000  /  26220
load  25000  /  26220
load  26000  /  26220


In [2]:

import torch

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# 显存不够可以把 batch size 改小点
train_loader = DataLoader(train_dataset, batch_size=3072, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False, num_workers=0)

In [3]:
import torch

class InceptionA(torch.nn.Module):
    def __init__(self, in_ch) -> None:
        super().__init__()
        self.branch_1x1 = torch.nn.Conv2d(in_ch, 16, kernel_size=1)

        self.branch_5x5_1 = torch.nn.Conv2d(in_ch, 16, kernel_size=1)
        self.branch_5x5_2 = torch.nn.Conv2d(16, 24, kernel_size=5, padding=2)

        self.branch_3x3_1 = torch.nn.Conv2d(in_ch, 16, kernel_size=1)
        self.branch_3x3_2 = torch.nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch_3x3_3 = torch.nn.Conv2d(24, 24, kernel_size=3, padding=1)

        self.branch_pool = torch.nn.Conv2d(in_ch, 24, kernel_size=1)

    def forward(self, x):
        branch_1x1 = self.branch_1x1(x)

        branch_5x5 = self.branch_5x5_1(x)
        branch_5x5 = self.branch_5x5_2(branch_5x5)

        branch_3x3 = self.branch_3x3_1(x)
        branch_3x3 = self.branch_3x3_2(branch_3x3)
        branch_3x3 = self.branch_3x3_3(branch_3x3)

        branch_pool = torch.nn.functional.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch_1x1, branch_5x5, branch_3x3, branch_pool] # 16 + 24 + 24 + 24
        return torch.cat(outputs, 1)


class GoogleNet(torch.nn.Module):
    def __init__(self, channels) -> None:
        super().__init__()
        self.conv1 = torch.nn.Conv2d(channels, 10, kernel_size=5)
        self.conv2 = torch.nn.Conv2d(88, 20, kernel_size=5)
        self.incep1 = InceptionA(10)
        self.incep2 = InceptionA(20)
        self.mp = torch.nn.MaxPool2d(2)
        self.fc = torch.nn.Linear(38808, 4)

    def forward(self, x):
        in_size = x.size(0)
        x = torch.nn.functional.relu(self.mp(self.conv1(x)))
        x = self.incep1(x)
        x = torch.nn.functional.relu(self.mp(self.conv2(x)))
        x = self.incep2(x)
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x


In [4]:

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
if not use_cuda:
    print("WARNING: CPU will be used for training.")

model = GoogleNet(3).to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [5]:
import time

start_time = time.time()
def train(epoch):
    global start_time
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
    cur_time = time.time()
    cost = cur_time - start_time
    print(f'Train Epoch: {epoch}, Loss: {loss.item():.8f}, cost: {cost:.2f} s')
    start_time = cur_time
            
def test():
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    acc = 100. * correct / len(test_loader.dataset)

    print(f'=== Test: Loss: {test_loss:.8f}, Acc: {acc:.4f} ===')
    return test_loss, acc

In [6]:
if use_cuda:
    torch.cuda.empty_cache()

output = Path('checkpoints')
output.mkdir(exist_ok=True, parents=True)

model_name = model.__class__.__name__
best_model_path = output / f'{model_name}_best.pt'

best_epoch = 0
best_loss = 100.0
best_acc = 0.0
default_interval = 10

def pipeline(start_epoch = 0, test_interval = default_interval):
    global best_epoch, best_loss, best_acc

    for epoch in range(start_epoch, 1000):
        train(epoch)
        if epoch % test_interval != 0:
            continue
        
        loss, acc = test()
        print(f'=== Pre best is {best_epoch}, Loss: {best_loss:.8f}, Acc: {best_acc:.4f} ===')
        torch.save(model, output / f'{model_name}_{epoch}.pt')
        if loss > best_loss:
            if epoch - best_epoch > test_interval * 10:
                print('No improvement for a long time, Early stop!')
                break
            else:
                continue
        best_epoch = epoch
        best_loss = loss
        best_acc = acc
        print(f'====== New best is {best_epoch}, Loss: {best_loss:.8f}, Acc: {best_acc:.4f} ======')
        torch.save(model, best_model_path)

pipeline()

Train Epoch: 0, Loss: 1.60028446, cost: 5.70 s
=== Test: Loss: 0.00319802, Acc: 45.5759 ===
=== Pre best is 0, Loss: 100.00000000, Acc: 0.0000 ===
====== New best is 0, Loss: 0.00319802, Acc: 45.5759 ======
Train Epoch: 1, Loss: 1.28577912, cost: 3.73 s
Train Epoch: 2, Loss: 1.28855324, cost: 3.34 s
Train Epoch: 3, Loss: 1.25775635, cost: 3.35 s
Train Epoch: 4, Loss: 1.24128604, cost: 3.35 s
Train Epoch: 5, Loss: 1.13937271, cost: 3.36 s
Train Epoch: 6, Loss: 1.02353418, cost: 3.36 s
Train Epoch: 7, Loss: 0.84396064, cost: 3.37 s
Train Epoch: 8, Loss: 0.58244020, cost: 3.39 s
Train Epoch: 9, Loss: 0.50752330, cost: 3.39 s
Train Epoch: 10, Loss: 0.45972630, cost: 3.37 s
=== Test: Loss: 0.00098966, Acc: 81.9222 ===
=== Pre best is 0, Loss: 0.00319802, Acc: 45.5759 ===
====== New best is 10, Loss: 0.00098966, Acc: 81.9222 ======
Train Epoch: 11, Loss: 0.43363693, cost: 3.77 s
Train Epoch: 12, Loss: 0.40060765, cost: 3.38 s
Train Epoch: 13, Loss: 0.36842823, cost: 3.38 s
Train Epoch: 14, L

KeyboardInterrupt: 

In [7]:
model = torch.load(best_model_path)
test()

=== Test: Loss: 0.00011718, Acc: 98.8749 ===


(0.00011718434812309267, 98.87490465293669)

In [9]:
# 导出 onnx

import torch.onnx
from pathlib import Path


def convert_onnx(path: Path):
    model = torch.load(path, map_location=torch.device("cpu"))
    model.eval()
    dummy_input = torch.randn(1, 3, 96, 96)
    torch.onnx.export(
        model,
        dummy_input,
        path.with_suffix(".onnx"),
        input_names=["input"],
        output_names=["output"],
    )


convert_onnx(best_model_path)


In [10]:
import onnx

onnx.checker.check_model(str(best_model_path.with_suffix(".onnx")))